In [1]:
!pip install joblib

In [2]:
import joblib

In [ ]:
"""
to_save = {"knn" : knn,
           "decision_tree": best_dt,
           "logistic_regression": log_reg,
           "X_train": X_train} # can add the data if wanted

filename = "class_algos.joblib"

joblib.dump(to_save, filename)
"""

In [ ]:
"""
saved = joblib.load(filename)
dt_saved = saved["decision_tree"]
dt_saved.predict(X_test)
"""

SyntaxError: invalid syntax (2026104101.py, line 2)